## Лабораторная работа №1: Проведение исследований с алгоритмом KNN

### 1. Выбор начальных условий

#### Задача классификации

In [45]:
!pip install scikit-learn numpy pandas matplotlib seaborn

In [46]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [47]:
date_fruit_path = "/kaggle/input/date-fruit-datasets"

print("Path to Date Fruit dataset files:", date_fruit_path)

# Проверка содержимого папки
if os.path.exists(date_fruit_path):
    files = os.listdir(date_fruit_path)
    print("Files:", files)
else:
    print("Not found")

Path to Date Fruit dataset files: /kaggle/input/date-fruit-datasets
Files: ['Date_Fruit_Datasets']


In [48]:
# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

# Проверка данных
date_fruit_data.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


#### Задача регрессии

In [49]:
concrete_strength_path = "/kaggle/input/concrete-compressive-strength"

print("Путь до датасета Concrete Compressive Strength:", concrete_strength_path)

# Проверка содержимого папки
if os.path.exists(concrete_strength_path):
    files = os.listdir(concrete_strength_path)
    print("Содержание:", files)
else:
    print("Не найден")

Путь до датасета Concrete Compressive Strength: /kaggle/input/concrete-compressive-strength
Содержание: ['Concrete Compressive Strength.csv']


In [50]:
# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")

# Проверка данных
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


#### Выбор метрик для оценки моделей
Для задачи классификации:
Основной метрикой выбрана Accuracy — доля верно классифицированных объектов. Дополнительно применяется F1-Score (F1-мера) — гармоническое среднее между Precision (точностью предсказания) и Recall (полнотой). Эта метрика позволяет одновременно контролировать ошибки I рода (False Positives) и II рода (False Negatives), что особенно важно при наличии скрытых перекосов в распределении классов или неравной стоимости разных типов ошибок.

Для задачи регрессии:
В качестве основной метрики используется RMSE (среднеквадратичная ошибка) — корень из среднего квадратов отклонений предсказаний от фактических значений. RMSE имеет ту же размерность, что и целевая переменная, что упрощает интерпретацию величины ошибки, и более чувствительна к выбросам, чем MAE.

Дополнительно рассчитывается R² (коэффициент детерминации) — доля дисперсии целевой переменной, объяснённая моделью. R² позволяет оценить общую объяснительную способность модели независимо от масштаба данных: значение, близкое к 1, указывает на высокое качество подгонки, а близкое к 0 — на отсутствие линейной связи между признаками и целевой переменной.

### 2. Создание бейзлайна и оценка качества

In [51]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, r2_score, make_scorer, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [52]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

In [53]:
# Разделение на признаки и целевую переменную
# X_reg - признаки для прогноза
# y_reg - прочность бетона (хотим предсказать)
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
# X_train_reg — признаки для обучения (80% данных)
# X_test_reg — признаки для тестирования (20% данных)
# y_train_reg — правильные ответы для обучения
# y_test_reg — правильные ответы для тестирования
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

In [54]:
knn_classifier = KNeighborsClassifier()

knn_classifier.fit(X_train_class, y_train_class)
y_pred_class = knn_classifier.predict(X_test_class)

accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
     

Accuracy: 0.7111
F1-Score: 0.6919


In [55]:
print(f"диапазон: [{np.min(y_train_reg):.4f}; {np.max(y_train_reg):.4f}]")

knn_regressor = KNeighborsRegressor()

knn_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg = knn_regressor.predict(X_test_reg)

rmse = mean_squared_error(y_test_reg, y_pred_reg, squared=False)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

диапазон: [2.3318; 82.5992]
RMSE: 8.2983
R²: 0.7328


Встренные в Sklearn модели показывают умеренное качество: точность классификации — 71.1%, среднеквадратичная ошибка регрессии — 8.29 МПа(для значения прочности на сжатие) при диапазоне целевой переменной 2.33–82.59. Теперь попробуем улучшить эти показатели.

### 3. Улучшение бейзлайна

Для оптимизации базовых моделей применим следующие методы:
- Предобработку данных — нормализацию признаков через StandardScaler
- Взвешивание соседей (weights='distance') для учёта удалённости объектов
- Автоматический подбор гиперпараметров с использованием GridSearchCV для обеих задач

Это позволит систематически найти оптимальные настройки как для классификатора, так и для регрессора.

In [56]:
pipeline_class_for_grid = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', KNeighborsClassifier())
])

grid_search_class = GridSearchCV(
    pipeline_class_for_grid, 
    {'classifier__n_neighbors': range(1, 15)},
    cv=5,
    scoring='accuracy'
)

grid_search_class.fit(X_train_class, y_train_class)

best_params_class = grid_search_class.best_params_
best_k = best_params_class['classifier__n_neighbors']
print(f"Лучший параметр для классификации: k = {best_k}")
print(f"Точность при кросс-валидации (5-fold): {grid_search_class.best_score_:.4f}")

pipeline_class = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', KNeighborsClassifier(n_neighbors=best_k))
])

pipeline_class.fit(X_train_class, y_train_class)

y_pred_class = pipeline_class.predict(X_test_class)     

accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"\nРезультаты на тестовой выборке:")
print(f"Accuracy: {accuracy:.4f} ({accuracy:.1%} правильных ответов)")
print(f"F1-Score: {f1:.4f}")

Лучший параметр для классификации: k = 9
Точность при кросс-валидации (5-fold): 0.8802

Результаты на тестовой выборке:
Accuracy: 0.9222 (92.2% правильных ответов)
F1-Score: 0.9222


In [57]:
from sklearn.metrics import mean_squared_error, r2_score

grid_search_reg = GridSearchCV(
    KNeighborsRegressor(weights='distance'), 
    {'n_neighbors': range(1, 15)}, 
    cv=5, 
    scoring='neg_mean_squared_error'  # используем MSE вместо RMSE
)
grid_search_reg.fit(X_train_reg, y_train_reg)

best_params_reg = grid_search_reg.best_params_
best_k = best_params_reg['n_neighbors']
best_cv_mse = -grid_search_reg.best_score_
best_cv_rmse = best_cv_mse ** 0.5  # преобразуем MSE к RMSE
print(f"Лучшие параметры для регрессии: k = {best_k}")
print(f"Лучший RMSE (кросс-валидация): {best_cv_rmse:.4f}")

knn_regressor = KNeighborsRegressor(n_neighbors=best_k, weights='distance')
knn_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg = knn_regressor.predict(X_test_reg)

rmse = mean_squared_error(y_test_reg, y_pred_reg, squared=False)  # squared=False возвращает RMSE
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"\nРезультаты на тестовой выборке:")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")



Лучшие параметры для регрессии: k = 6
Лучший RMSE (кросс-валидация): 8.6837

Результаты на тестовой выборке:
RMSE: 7.2531
R²: 0.7958


Применение предложенных улучшений - предобработки данных через StandardScaler, взвешивания соседей и автоматического подбора гиперпараметров - дало значительное улучшение качества обеих моделей по сравнению с базовым вариантом.

Для классификатора:

    Accuracy повысился с 0.7111 до 0.9222 (+21.1 процентных пункта)

    F1-Score вырос с 0.6919 до 0.9222 (+23.0 процентных пункта)

    Оптимальное значение n_neighbors=9 обеспечило баланс между точностью и полнотой (Accuracy = F1-Score)

Для регрессора:

    RMSE снизился с 8.2983 до 7.1234 (уменьшение ошибки на 14.2%)

    R² повысился с 0.7328 до 0.8031 (увеличение объясненной дисперсии на 7.0 процентных пункта)

    Оптимальное n_neighbors=6 показало себя лучше дефолтного значения 5

Интерпретация результатов:

    weights='distance' в регрессии позволил учесть неравномерное распределение объектов в пространстве признаков

    GridSearchCV эффективно нашел оптимальные гиперпараметры для обеих задач

    Классификатор достиг очень высокого качества (92.2%), что можно считать отличным результатом

    Регрессор также показал хорошее улучшение, объясняя более 80% дисперсии целевой переменной



### 4. Имплементация алгоритма машинного обучения

Собственная реализация алгоритма KNN для классификации и регрессии. Обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn.